In [1]:
import numpy as np

In [37]:
# First, because I'm missing a set of quality input data. So lets find a way to generate some.
# I will use player number as their skill, just because it's easy. I don't want it completely
# random because then what is the model learning, really?
players = np.arange(1, 11)
maps = ["The Skeld", "MIRA HQ", "Polus", "Airship"]
no_impostors = 2
confirm_ejects = False
emergency_meetings = 1
emergency_cooldown = 20
discussion_time = 15
voting_time = 120
anonymous_votes = True
player_speed = 1.25
crewmate_vision = 0.75
impostor_vision = 1.5
kill_cooldown = 35.0
kill_distance = "Short"
visual_tasks = False
task_bar_updates = "Meetings"
common_tasks = 2
long_tasks = 3
short_tasks = 5

In [35]:
# My plan for generating match results involves picking
# two players as impostors, then summing the player numbers
# to get a 'skill value', then generate a random number
# based on these skill values to get a winner

# I want the skill values to be about equal for the most part
def skill_values(rounds):
    imp_sum = 0
    crew_sum = 0
    for _ in range(rounds):
        impostors = rng.choice(players, no_impostors, replace=False)
        crewmates = np.setdiff1d(players, impostors, assume_unique=True)
        imp_sum += np.sum(impostors)
        crew_sum += np.sum(crewmates)
    print('Average crew skill:', crew_sum / rounds)
    print('Average impostor skill:', imp_sum / rounds)
    print('Ratio:', crew_sum / imp_sum)
    
skill_values(1000)
# As expected, it is close enough to 4x
# I must have just got unlucky with my testing earlier and had a big
# string of impostor wins

Average crew skill: 43.923
Average impostor skill: 11.077
Ratio: 3.965243296921549


In [39]:
# For now I will use the same settings for all matches
rng = np.random.default_rng()
def get_winner(crewmates, impostors):
    impostor_skill = np.sum(impostors) * 4
    crewmate_skill = np.sum(crewmates)
    match_value = rng.integers(impostor_skill + crewmate_skill)
    return 'crewmates' if match_value < crewmate_skill else 'impostors'

def generate_match():
    played_map = "Airship"
    impostors = rng.choice(players, no_impostors, replace=False)
    crewmates = np.setdiff1d(players, impostors, assume_unique=True)
    winner = get_winner(crewmates, impostors)
    stradd = np.core.defchararray.add
    crew_names = list(stradd('Player ', crewmates.astype(str)))
    imp_names = list(stradd('Player ', impostors.astype(str)))
    match_result = {
        'crewmates': crew_names,
        'impostors': imp_names,
        'map': played_map,
        'confirm_ejects': confirm_ejects,
        'emergency_meetings': emergency_meetings,
        'emergency_cooldown': emergency_cooldown,
        'discussion_time': discussion_time,
        'voting_time': voting_time,
        'anonymous_votes': anonymous_votes,
        'player_speed': player_speed,
        'crewmate_vision': crewmate_vision,
        'impostor_vision': impostor_vision,
        'kill_cooldown': kill_cooldown,
        'kill_distance': kill_distance,
        'visual_tasks': visual_tasks,
        'task_bar_updates': task_bar_updates,
        'common_tasks': common_tasks,
        'long_tasks': long_tasks,
        'short_tasks': short_tasks,
        'match_winner': winner
    }
    return match_result
    
generate_match()

{'crewmates': ['Player 1',
  'Player 2',
  'Player 4',
  'Player 5',
  'Player 6',
  'Player 7',
  'Player 9',
  'Player 10'],
 'impostors': ['Player 3', 'Player 8'],
 'map': 'Airship',
 'confirm_ejects': False,
 'emergency_meetings': 1,
 'emergency_cooldown': 20,
 'discussion_time': 15,
 'voting_time': 120,
 'anonymous_votes': True,
 'player_speed': 1.25,
 'crewmate_vision': 0.75,
 'impostor_vision': 1.5,
 'kill_cooldown': 35.0,
 'kill_distance': 'Short',
 'visual_tasks': False,
 'task_bar_updates': 'Meetings',
 'common_tasks': 2,
 'long_tasks': 3,
 'short_tasks': 5,
 'match_winner': 'impostors'}

In [42]:
# Generate a bunch of matches, then count how many times the
# players won to get a general sense of if the actual match
# results align with what I was intending
match_count = 1000
matches = [generate_match() for _ in range(match_count)]
wins = {f'Player {p}': 0 for p in players}
for match in matches:
    # ngl this is a bit of a happy accident
    # I wasn't intending to set it up so I could access the winning
    # players like this so easily
    for p in match[match['match_winner']]:
        wins[p] += 1
        
wins

{'Player 1': 465,
 'Player 2': 465,
 'Player 3': 491,
 'Player 4': 497,
 'Player 5': 525,
 'Player 6': 512,
 'Player 7': 519,
 'Player 8': 518,
 'Player 9': 541,
 'Player 10': 515}

There is a bit of progression, player 1 is definitely performing worse than player 10  
I think I would have liked a bigger difference because this data is randomly generated, so the patterns just won't be there as much.

That being said though, this will really be good enough for now. I'm sure any model I produce will have a better than 50% accuracy based on this

In [44]:
import json
with open('data/matches.json', 'w') as outfile:
    json.dump(matches, outfile)